In [1]:
from pyts.datasets import load_basic_motions
import numpy as np

In [2]:
bunch = load_basic_motions()
X_train, X_test, y_train, y_test = load_basic_motions(return_X_y=True)

In [3]:
X_train.shape

(40, 6, 100)

In [4]:
class reservoir_features:
    '''
    creates an object associated with a multivariate dataset
    
    
    '''
    def __init__(self,data,num_features):
        '''
        data: Takes in a multidimensional array (x * y * z) - z>y
        Initializes it
        x: Timeseries
        y: Attributes for a given timeseries observation
        z: timestamped observations (features)
        
        num_features: you must specify the dimension you want to reduce it to
        
        '''
        self.features = []
        self.filters_used = []
        self.original_data = data.copy()
        self.data = data.copy()
        self.num_features = num_features
        self.x = data.shape[0]
        self.y = data.shape[1]
        self.z = data.shape[2]
        # perform checks 
        #1. 3d numpy array
        #2. Each time series should have same number of observations
        #3. num_features should be less than timestamped observations
    
    def normalize(self):
        '''
        Each attribute could potentially be on a different scale
        modifies the original data and performs a min max normalization
        '''
        for i in range(self.original_data.shape[0]):
            for j in range(self.original_data.shape[1]):
                self.data[i][j] = (self.original_data[i][j] - self.original_data[i][j].min())/(self.original_data[i][j].max()-self.original_data[i][j].min())
    
    
    def filters(self,stride_len = [1], num_filters = 1):
        '''
        stride_len: num of columns to skip after each filter multiplication
        num_filters: you can specify the number of filters you need; each filter will be of a differnt size
        size of filter = n*m 
        (n = # of rows = attribute size, 
        m = # of columns)
        '''
        #Have error check to make sure stride len is a list and value is <length of attributes
        n = self.y
        
        
        #Edge case vals is empty/smaller than num_filters
        
        for iteration in range(num_filters):
            m = self._get_m(stride_len[iteration])
            filter_a = np.random.random((n,m))
            print("filter of size ", str(n), "*", str(m), "was created\n")
            self.filters_used.append(filter_a)
            
            temp_features =[]
            for i in range(self.x):
                temp = []
                j = 0
                while j + m < self.data.shape[2]:
                    temp.append((filter_a*self.data[i,:,j:j+m]).mean())
                    j+=stride_len[iteration]
                temp_features.append(temp)
            self.features.append(temp_features)
    
    
    def _get_m(self,stride_len):
        '''
        stride_len: 
        based on stride length,& num_features, we calculate possible filter size 
        '''
        m = self.z -(self.num_features)*stride_len
        return m
    
    
    def result_features(self):
        '''
        if multiple filters were added, takes the average result
        '''
        ans =[]
        for timeseries in range(len(self.features[0])):
            temp =[]
            for feature in range(len(self.features[0][0])):
                val = np.mean([self.features[filter][timeseries][feature] for filter in range(len(self.features))])
                temp.append(val)
            ans.append(temp)
        return ans
        

In [5]:
#create object
a = reservoir_features(X_train,num_features = 18)

In [6]:
#normalize
a.normalize()

In [7]:
#create 2 filters
a.filters(stride_len = [1,2], num_filters = 2)

filter of size  6 * 82 was created

filter of size  6 * 64 was created



In [8]:
len(a.features)

2

In [9]:
a.filters(stride_len = [3], num_filters = 1)

filter of size  6 * 46 was created



In [10]:
len(a.features)

3

In [11]:
ans = a.result_features()

In [12]:
len(ans)

40

In [13]:
len(ans[0])

18

## Validation on 7 datasets using logistic regression and SVM models

### Import the libraries

In [14]:
# import datsets
from sktime.datasets import load_UCR_UEA_dataset
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy
from pyts.datasets import load_basic_motions

In [24]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

### Run Logistic Regression on the Transformed Output

In [92]:
#create object
X_train_transformed = reservoir_features(X_train,num_features = 16)
#normalize
X_train_transformed.normalize()
#create 2 filters
X_train_transformed.filters(stride_len = [3], num_filters = 1)
X_train_reservoir_transformed = X_train_transformed.result_features()
print(len(X_train_reservoir_transformed))
print(len(X_train_reservoir_transformed[0]))

filter of size  6 * 52 was created

40
16


In [93]:
#create object
X_test_transformed = reservoir_features(X_test,num_features = 16)
#normalize
X_test_transformed.normalize()
#create 2 filters
X_test_transformed.filters(stride_len = [3], num_filters = 1)
X_test_reservoir_transformed = X_test_transformed.result_features()
print(len(X_test_reservoir_transformed))
print(len(X_test_reservoir_transformed[0]))

filter of size  6 * 52 was created

40
16


In [94]:
print(set(y_train))
print(set(y_test))

{'Walking', 'Badminton', 'Standing', 'Running'}
{'Walking', 'Badminton', 'Standing', 'Running'}


build a l2-regularized logistic regression model

In [95]:
l2_model = LogisticRegression(random_state=0, max_iter=100, multi_class='multinomial', solver='lbfgs', C=1.0).fit(X_train_reservoir_transformed, y_train)

In [96]:
l2_model.score(X_train_reservoir_transformed, y_train)

0.45

In [97]:
y_train_pred = l2_model.predict(X_train_reservoir_transformed)
l2_train_acc = accuracy_score(y_train, y_train_pred)
print(l2_train_acc)

0.45


In [98]:
y_pred = l2_model.predict(X_test_reservoir_transformed)

In [99]:
l2_test_acc = accuracy_score(y_test, y_pred)
print(l2_test_acc)

0.35


write a function for data transformation and baseline l2 logistic regression validation

In [15]:
def transform_data(data, num_features, stride_len, num_filters):
    data_transformed = reservoir_features(data ,num_features = num_features)
    #normalize
    data_transformed.normalize()
    #create 2 filters
    data_transformed.filters(stride_len = stride_len, num_filters = num_filters)
    data_transformed = data_transformed.result_features()
    return data_transformed

In [16]:
def run_baseline_l2_logistic_regression(X_train, y_train, X_test, y_test):
    l2_model = LogisticRegression(random_state=0, max_iter=100, multi_class='multinomial', solver='lbfgs', C=1.0).fit(X_train, y_train)
    # training metrics
    print('train_acc: %f '%l2_model.score(X_train, y_train))
    train_auc = roc_auc_score(y_train, l2_model.predict_proba(X_train), multi_class='ovr')
    print('train_AUC: %f '%train_auc)
    # testing metrics
    y_pred = l2_model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print('test_acc: %f '%test_acc)
    test_auc = roc_auc_score(y_test, l2_model.predict_proba(X_test), multi_class='ovr')
    print('test_AUC: %f '%test_auc)

In [102]:
X_train_BM, X_test_BM, y_train_BM, y_test_BM = load_basic_motions(return_X_y=True)

In [103]:
X_train_BM = transform_data(X_train_BM, 16, [3], 1)
X_test_BM = transform_data(X_test_BM, 16, [3], 1)

filter of size  6 * 52 was created

filter of size  6 * 52 was created



In [104]:
run_baseline_l2_logistic_regression(X_train_BM, y_train_BM, X_test_BM, y_test_BM)

train_acc: 0.450000 
train_AUC: 0.712500 
test_acc: 0.350000 
test_AUC: 0.672500 


### Run SVM on the Transformed Output
For multi-class classification, try both the one_vs_one(ovo) method and the one_vs_rest(ovr) method

run the baseline SVM model using the one_vs_one method on the basic_motions dataset

In [105]:
one_vs_one_svc_clf = svm.SVC(C=1.0, decision_function_shape='ovo')
one_vs_one_svc_clf.fit(X_train_BM, y_train_BM)

SVC(decision_function_shape='ovo')

In [106]:
y_train_pred_BM = one_vs_one_svc_clf.predict(X_train_BM)
ovo_svc_train_clf_acc = accuracy_score(y_train_BM, y_train_pred_BM)
print(ovo_svc_train_clf_acc)

0.65


In [107]:
y_pred_BM = one_vs_one_svc_clf.predict(X_test_BM)
ovo_svc_test_clf_acc = accuracy_score(y_test_BM, y_pred_BM)
print(ovo_svc_test_clf_acc)

0.35


run the baseline SVM model using the one_vs_rest method on the basic_motions dataset

In [108]:
one_vs_rest_svc_clf = svm.SVC(decision_function_shape='ovr')
one_vs_rest_svc_clf.fit(X_train_BM, y_train_BM)
# 4 classes

SVC()

In [109]:
y_train_pred_BM = one_vs_rest_svc_clf.predict(X_train_BM)
ovr_svc_train_clf_acc = accuracy_score(y_train_BM, y_train_pred_BM)
print(ovr_svc_train_clf_acc)

0.65


In [110]:
y_pred_BM = one_vs_rest_svc_clf.predict(X_test_BM)
ovr_svc_test_clf_acc = accuracy_score(y_test_BM, y_pred_BM)
print(ovr_svc_test_clf_acc)

0.35


Write a framework for parameter tuning on SVM models <br>
SVM parameters: C, kernel, degree (polynomial only), decision_function, break_ties

In [18]:
def get_svc_acc(X_train, y_train, X_test, y_test, C_select, kernel_select, d_select, df_select):
    model = svm.SVC(C=C_select, kernel=kernel_select, degree=d_select, decision_function_shape=df_select, probability=True).fit(X_train, y_train)
    # training metrics
    train_pred = model.predict(X_train)
    train_acc = accuracy_score(y_train, train_pred)
    if len(set(y_train)) == 2:
        train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:,1], multi_class='ovo')
    else:
        train_auc = roc_auc_score(y_train, model.predict_proba(X_train), multi_class='ovr')
    train_f1 = f1_score(y_train, train_pred, average='weighted')
    # testing metrics
    test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, test_pred)
    if len(set(y_train)) == 2:
        test_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1], multi_class='ovo')
    else:
        test_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
    test_f1 = f1_score(y_test, test_pred, average='weighted')
    return 'train_acc: %f '%train_acc + 'train_auc: %f '%train_auc + 'train_f1: %f '%train_f1 + 'test_acc: %f '%test_acc + 'test_auc: %f '%test_auc + 'test_f1: %f '%test_f1

In [19]:
def write_SVM_CV_output(f_name, C_list, kernel_list, d_list, df_list, X_train, y_train, X_test, y_test):
    with open(f_name, 'w') as f:
        for C in C_list:
            for kernel in kernel_list:
                for df in df_list:
                    if kernel == 'poly':
                        for d in d_list:
                            f.write('C = %f'%C)
                            f.write('\n')
                            f.write('kernel = polynomial')
                            f.write('\n')
                            f.write('decision_function = '+df)
                            f.write('\n')
                            f.write('degree = %d'%d)
                            f.write('\n')
                            f.write(get_svc_acc(X_train, y_train, X_test, y_test, C, kernel, d, df))
                            f.write('\n')
                            f.write('\n')
                    else:
                        f.write('C = %f'%C)
                        f.write('\n')
                        f.write('kernel = '+kernel)
                        f.write('\n')
                        f.write('decision_function = '+df)
                        f.write('\n')
                        f.write(get_svc_acc(X_train, y_train, X_test, y_test, C, kernel, 0, df))
                        f.write('\n')
                        f.write('\n')
    f.close()

In [113]:
# output CV result for basic_motions dataset
C_list = [1.0, 0.1, 0.01, 0.001]
kernel_list = ['linear', 'rbf', 'poly', 'sigmoid']
d_list = [2, 3, 4]
df_list = ['ovr', 'ovo']
write_SVM_CV_output('basic_motions_svm_cv_results.txt', C_list, kernel_list, d_list, df_list, X_train_BM, y_train_BM, X_test_BM, y_test_BM)

In [20]:
def find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train, y_train, X_test, y_test):
    best_test_acc = None
    best_test_auc = None
    best_test_f1 = None
    best_acc_result_list = []
    best_auc_result_list = []
    best_f1_result_list = []
    for C in C_list:
        for kernel in kernel_list:
            for df in df_list:
                if kernel == 'poly':
                    for d in d_list:
                        model = svm.SVC(C=C, kernel=kernel, degree=d, decision_function_shape=df, probability=True).fit(X_train, y_train)
                        # training metrics
                        train_pred = model.predict(X_train)
                        train_acc = accuracy_score(y_train, train_pred)
                        if len(set(y_train)) == 2:
                            train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:,1], multi_class='ovo')
                        else:
                            train_auc = roc_auc_score(y_train, model.predict_proba(X_train), multi_class='ovr')
                        train_f1 = f1_score(y_train, train_pred, average='weighted')
                        # testing metrics
                        test_pred = model.predict(X_test)
                        test_acc = accuracy_score(y_test, test_pred)
                        if len(set(y_train)) == 2:
                            test_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1], multi_class='ovo')
                        else:
                            test_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
                        test_f1 = f1_score(y_test, test_pred, average='weighted')
                        if not best_test_acc or best_test_acc <= test_acc:
                            best_test_acc = test_acc
                            best_acc_result_list = [C, kernel, d, df, train_acc, test_acc]
                        if not best_test_auc or best_test_auc <= test_auc:
                            best_test_auc = test_auc
                            best_auc_result_list = [C, kernel, d, df, train_auc, test_auc] 
                        if not best_test_f1 or best_test_f1 <= test_f1:
                            best_test_f1 = test_f1
                            best_f1_result_list = [C, kernel, d, df, train_f1, test_f1]
                else:
                    model = svm.SVC(C=C, kernel=kernel, decision_function_shape=df, probability=True).fit(X_train, y_train)
                    # training metrics
                    train_pred = model.predict(X_train)
                    train_acc = accuracy_score(y_train, train_pred)
                    if len(set(y_train)) == 2:
                        train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:,1], multi_class='ovo')
                    else:
                        train_auc = roc_auc_score(y_train, model.predict_proba(X_train), multi_class='ovr')
                    train_f1 = f1_score(y_train, train_pred, average='weighted')
                    # testing metrics
                    test_pred = model.predict(X_test)
                    if len(set(y_train)) == 2:
                        test_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1], multi_class='ovo')
                    else:
                        test_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
                    test_acc = accuracy_score(y_test, test_pred)
                    test_f1 = f1_score(y_test, test_pred, average='weighted')
                    if not best_test_acc or best_test_acc <= test_acc:
                        best_test_acc = test_acc
                        best_acc_result_list = [C, kernel, 0, df, train_acc, test_acc]
                    if not best_test_auc or best_test_auc <= test_auc:
                        best_test_auc = test_auc
                        best_auc_result_list = [C, kernel, 0, df, train_auc, test_auc]
                    if not best_test_f1 or best_test_f1 <= test_f1:
                        best_test_f1 = test_f1
                        best_f1_result_list = [C, kernel, 0, df, train_f1, test_f1]
    return (best_acc_result_list, best_auc_result_list, best_f1_result_list)

In [115]:
find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train_BM, y_train_BM, X_test_BM, y_test_BM)

([1.0, 'poly', 2, 'ovo', 0.95, 0.5],
 [1.0, 'poly', 2, 'ovr', 0.9916666666666667, 0.72],
 [1.0, 'poly', 2, 'ovo', 0.949874686716792, 0.4825274725274725])

# Validation using UCR_UEA Datasets
There are six total UCR_UEA datasets - ArticularyWordRecognition, AtrialFibrillation, Cricket, Epilepsy, FingerMovements and Handwriting. <br>
For each dataset, we run validation using the baseline L2 logistic regression model and SVM models with parameter tuning.

### Validation using the ArticularyWordRecognition dataset

In [116]:
X_train_WR, y_train_WR = load_UCR_UEA_dataset('ArticularyWordRecognition', split="train", return_X_y=True)
print(X_train_WR.shape)
print(y_train_WR.shape)

(275, 9)
(275,)


In [117]:
X_test_WR, y_test_WR = load_UCR_UEA_dataset('ArticularyWordRecognition', split="test", return_X_y=True)
print(X_test_WR.shape)
print(y_test_WR.shape)

(300, 9)
(300,)


In [118]:
print(set(y_train_WR))

{'5.0', '24.0', '14.0', '6.0', '11.0', '15.0', '4.0', '20.0', '13.0', '19.0', '7.0', '1.0', '16.0', '10.0', '21.0', '9.0', '25.0', '8.0', '23.0', '17.0', '3.0', '12.0', '2.0', '18.0', '22.0'}


In [119]:
X_train_3d_WR = from_nested_to_3d_numpy(X_train_WR)
X_test_3d_WR = from_nested_to_3d_numpy(X_test_WR)

print(X_train_3d_WR.shape)
print(X_test_3d_WR.shape)

(275, 9, 144)
(300, 9, 144)


In [120]:
X_train_WR = transform_data(X_train_3d_WR, 40, [3], 1)
X_test_WR = transform_data(X_test_3d_WR, 40, [3], 1)

filter of size  9 * 24 was created

filter of size  9 * 24 was created



In [121]:
run_baseline_l2_logistic_regression(X_train_WR, y_train_WR, X_test_WR, y_test_WR)

train_acc: 0.458182 
train_AUC: 0.927851 
test_acc: 0.460000 
test_AUC: 0.913762 


In [122]:
C_list = [1.0, 0.1, 0.01, 0.001]
kernel_list = ['linear', 'rbf', 'poly', 'sigmoid']
d_list = [2,3,4]
df_list = ['ovr', 'ovo']

In [123]:
write_SVM_CV_output('word_recognition_svm_cv_results.txt', C_list, kernel_list, d_list, df_list, X_train_WR, y_train_WR, X_test_WR, y_test_WR)

In [124]:
find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train_WR, y_train_WR, X_test_WR, y_test_WR)

([1.0, 'rbf', 0, 'ovo', 0.7163636363636363, 0.5166666666666667],
 [0.1, 'poly', 3, 'ovr', 0.9956887052341599, 0.946701388888889],
 [1.0, 'rbf', 0, 'ovo', 0.7055725943722991, 0.4807748030824627])

### Validation using the AtrialFibrillation dataset

In [125]:
X_train_AF, y_train_AF = load_UCR_UEA_dataset('AtrialFibrillation', split="train", return_X_y=True)
print(X_train_AF.shape)
print(y_train_AF.shape)

(15, 2)
(15,)


In [126]:
X_test_AF, y_test_AF = load_UCR_UEA_dataset('AtrialFibrillation', split="test", return_X_y=True)
print(X_test_AF.shape)
print(y_test_AF.shape)

(15, 2)
(15,)


In [127]:
print(y_train_AF)

['n' 'n' 'n' 'n' 'n' 's' 's' 's' 's' 's' 't' 't' 't' 't' 't']


In [128]:
X_train_3d_AF = from_nested_to_3d_numpy(X_train_AF)
X_test_3d_AF = from_nested_to_3d_numpy(X_test_AF)

print(X_train_3d_AF.shape)
print(X_test_3d_AF.shape)

(15, 2, 640)
(15, 2, 640)


In [129]:
X_train_AF = transform_data(X_train_3d_AF, 40, [3], 1)
X_test_AF = transform_data(X_test_3d_AF, 40, [3], 1)

filter of size  2 * 520 was created

filter of size  2 * 520 was created



In [130]:
print(len(X_train_AF))
print(len(X_train_AF[0]))

15
40


In [131]:
run_baseline_l2_logistic_regression(X_train_AF, y_train_AF, X_test_AF, y_test_AF)

train_acc: 0.400000 
train_AUC: 0.660000 
test_acc: 0.266667 
test_AUC: 0.486667 


In [132]:
C_list = [1.0, 0.1, 0.01, 0.001]
kernel_list = ['linear', 'rbf', 'poly', 'sigmoid']
d_list = [2, 3, 4]
df_list = ['ovr', 'ovo']

In [133]:
write_SVM_CV_output('atrial_fibrillation_svm_cv_results.txt', C_list, kernel_list, d_list, df_list, X_train_AF, y_train_AF, X_test_AF, y_test_AF)

In [134]:
find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train_AF, y_train_AF, X_test_AF, y_test_AF)

([0.001, 'sigmoid', 0, 'ovo', 0.3333333333333333, 0.3333333333333333],
 [0.01, 'rbf', 0, 'ovo', 0.25333333333333335, 0.6466666666666667],
 [1.0, 'poly', 4, 'ovo', 1.0, 0.3333333333333333])

### Validation using the Cricket dataset

In [135]:
X_train_C, y_train_C = load_UCR_UEA_dataset('Cricket', split="train", return_X_y=True)
print(X_train_C.shape)
print(y_train_C.shape)

(108, 6)
(108,)


In [136]:
X_test_C, y_test_C = load_UCR_UEA_dataset('Cricket', split="test", return_X_y=True)
print(X_test_C.shape)
print(y_test_C.shape)

(72, 6)
(72,)


In [137]:
print(set(y_train_C))
print(set(y_test_C))

{'10.0', '12.0', '4.0', '5.0', '9.0', '2.0', '6.0', '11.0', '7.0', '1.0', '8.0', '3.0'}
{'10.0', '12.0', '4.0', '5.0', '9.0', '2.0', '6.0', '11.0', '7.0', '1.0', '8.0', '3.0'}


In [138]:
X_train_3d_C = from_nested_to_3d_numpy(X_train_C)
X_test_3d_C = from_nested_to_3d_numpy(X_test_C)

print(X_train_3d_C.shape)
print(X_test_3d_C.shape)

(108, 6, 1197)
(72, 6, 1197)


In [139]:
X_train_C = transform_data(X_train_3d_C, 200, [3], 1)
X_test_C = transform_data(X_test_3d_C, 200, [3], 1)

filter of size  6 * 597 was created

filter of size  6 * 597 was created



In [140]:
run_baseline_l2_logistic_regression(X_train_C, y_train_C, X_test_C, y_test_C)

train_acc: 0.388889 
train_AUC: 0.895062 
test_acc: 0.263889 
test_AUC: 0.861111 


In [141]:
C_list = [1.0, 0.1, 0.01, 0.001]
kernel_list = ['linear', 'poly', 'rbf', 'sigmoid']
d_list = [2, 3, 4]
df_list = ['ovr', 'ovo']

In [142]:
write_SVM_CV_output('cricket_svm_cv_results.txt', C_list, kernel_list, d_list, df_list, X_train_C, y_train_C, X_test_C, y_test_C)

In [143]:
find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train_C, y_train_C, X_test_C, y_test_C)

([1.0, 'poly', 2, 'ovo', 0.9074074074074074, 0.6388888888888888],
 [1.0, 'poly', 2, 'ovo', 0.9891507669285448, 0.943392255892256],
 [1.0, 'poly', 2, 'ovo', 0.90614608416466, 0.6153721278721278])

### Validation using the Epilepsy dataset

In [144]:
X_train_E, y_train_E = load_UCR_UEA_dataset('Epilepsy', split="train", return_X_y=True)
print(X_train_E.shape)
print(y_train_E.shape)

(137, 3)
(137,)


In [145]:
X_test_E, y_test_E = load_UCR_UEA_dataset('Epilepsy', split="test", return_X_y=True)
print(X_test_E.shape)
print(y_test_E.shape)

(138, 3)
(138,)


In [146]:
print(set(y_train_E))

{'epilepsy', 'running', 'walking', 'sawing'}


In [147]:
X_train_3d_E = from_nested_to_3d_numpy(X_train_E)
X_test_3d_E = from_nested_to_3d_numpy(X_test_E)

print(X_train_3d_E.shape)
print(X_test_3d_E.shape)

(137, 3, 206)
(138, 3, 206)


In [148]:
X_train_E = transform_data(X_train_3d_E, 16, [3], 1)
X_test_E = transform_data(X_test_3d_E, 16, [3], 1)

filter of size  3 * 158 was created

filter of size  3 * 158 was created



In [149]:
run_baseline_l2_logistic_regression(X_train_E, y_train_E, X_test_E, y_test_E)

train_acc: 0.299270 
train_AUC: 0.572439 
test_acc: 0.268116 
test_AUC: 0.509503 


In [150]:
# the SVM model using polynomial kernel for this dataset runs forever for degree=4, remove it
C_list = [1.0, 0.1, 0.01, 0.001]
kernel_list = ['linear', 'poly', 'rbf', 'sigmoid']
d_list = [2, 3]
df_list = ['ovr', 'ovo']

In [151]:
write_SVM_CV_output('epilepsy_svm_cv_results.txt', C_list, kernel_list, d_list, df_list, X_train_E, y_train_E, X_test_E, y_test_E)

In [152]:
find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train_E, y_train_E, X_test_E, y_test_E)

([1.0, 'poly', 3, 'ovo', 0.7372262773722628, 0.35507246376811596],
 [1.0, 'poly', 3, 'ovr', 0.7582429115642032, 0.591304260062147],
 [1.0, 'poly', 3, 'ovo', 0.7352078493069657, 0.3567753145423581])

### Validation using the FingerMovements dataset
Note that this is a binary classification task

In [153]:
X_train_FM, y_train_FM = load_UCR_UEA_dataset('FingerMovements', split="train", return_X_y=True)
print(X_train_FM.shape)
print(y_train_FM.shape)

(316, 28)
(316,)


In [154]:
X_test_FM, y_test_FM = load_UCR_UEA_dataset('FingerMovements', split="test", return_X_y=True)
print(X_test_FM.shape)
print(y_test_FM.shape)

(100, 28)
(100,)


In [155]:
X_FM, y_FM = load_UCR_UEA_dataset('FingerMovements', return_X_y=True)
print(X_FM.shape)
print(y_FM.shape)
X_train_FM, X_test_FM, y_train_FM, y_test_FM = train_test_split(X_FM, y_FM, test_size=0.2, random_state=42)
print(X_train_FM.shape)
print(y_train_FM.shape)
print(X_test_FM.shape)
print(y_test_FM.shape)

(416, 28)
(416,)
(332, 28)
(332,)
(84, 28)
(84,)


In [156]:
print(set(y_train_FM))

{'right', 'left'}


In [157]:
X_train_3d_FM = from_nested_to_3d_numpy(X_train_FM)
X_test_3d_FM = from_nested_to_3d_numpy(X_test_FM)

print(X_train_3d_FM.shape)
print(X_test_3d_FM.shape)

(332, 28, 50)
(84, 28, 50)


In [158]:
X_train_FM = transform_data(X_train_3d_FM, 16, [3], 1)
X_test_FM = transform_data(X_test_3d_FM, 16, [3], 1)

filter of size  28 * 2 was created

filter of size  28 * 2 was created



In [159]:
# The run_baseline_l2 function should not be run here since it is binary classification
l2_model_FM = LogisticRegression(random_state=0, max_iter=100, C=1.0).fit(X_train_FM, y_train_FM)
# training metrics
print('train_acc: %f '%l2_model_FM.score(X_train_FM, y_train_FM))
train_auc = roc_auc_score(y_train_FM, l2_model_FM.predict_proba(X_train_FM)[:, 1], multi_class='ovr')
print('train_AUC: %f '%train_auc)
# testing metrics
y_pred_FM = l2_model_FM.predict(X_test_FM)
test_acc = accuracy_score(y_test_FM, y_pred_FM)
print('test_acc: %f '%test_acc)
test_auc = roc_auc_score(y_test_FM, l2_model.predict_proba(X_test_FM)[:, 1], multi_class='ovr')
print('test_AUC: %f '%test_auc)


train_acc: 0.512048 
train_AUC: 0.556936 
test_acc: 0.440476 
test_AUC: 0.480161 


In [160]:
# only the ovo method is needed here
C_list = [1.0, 0.1, 0.01, 0.001]
kernel_list = ['linear', 'poly', 'rbf', 'sigmoid']
d_list = [2, 3]
df_list = ['ovo']

In [161]:
write_SVM_CV_output('finger_movements_svm_cv_results.txt', C_list, kernel_list, d_list, df_list, X_train_FM, y_train_FM, X_test_FM, y_test_FM)

In [162]:
find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train_FM, y_train_FM, X_test_FM, y_test_FM)

([0.01, 'poly', 2, 'ovo', 0.5903614457831325, 0.5714285714285714],
 [0.1, 'poly', 2, 'ovo', 0.6278377102175728, 0.5451408855664175],
 [0.01, 'poly', 2, 'ovo', 0.5863546851498658, 0.5626021575678326])

### Validation using the Handwriting Dataset

In [25]:
X_train_H, y_train_H = load_UCR_UEA_dataset('Handwriting', split="train", return_X_y=True)
print(X_train_H.shape)
print(y_train_H.shape)

(150, 3)
(150,)


In [26]:
X_test_H, y_test_H = load_UCR_UEA_dataset('Handwriting', split="test", return_X_y=True)
print(X_test_H.shape)
print(y_test_H.shape)

(850, 3)
(850,)


In [27]:
X_H, y_H = load_UCR_UEA_dataset('Handwriting', return_X_y=True)
print(X_H.shape)
print(y_H.shape)
X_train_H, X_test_H, y_train_H, y_test_H = train_test_split(X_H, y_H, test_size=0.5, random_state=42)
print(X_train_H.shape)
print(y_train_H.shape)
print(X_test_H.shape)
print(y_test_H.shape)

(1000, 3)
(1000,)
(500, 3)
(500,)
(500, 3)
(500,)


In [28]:
print(set(y_train_H))
print(set(y_test_H))

{'18.0', '7.0', '10.0', '9.0', '15.0', '21.0', '8.0', '12.0', '20.0', '4.0', '1.0', '25.0', '17.0', '3.0', '26.0', '23.0', '14.0', '6.0', '19.0', '24.0', '11.0', '2.0', '5.0', '16.0', '22.0', '13.0'}
{'18.0', '7.0', '10.0', '9.0', '15.0', '21.0', '8.0', '12.0', '4.0', '20.0', '1.0', '17.0', '25.0', '3.0', '26.0', '23.0', '14.0', '6.0', '19.0', '24.0', '11.0', '2.0', '5.0', '16.0', '22.0', '13.0'}


In [29]:
X_train_3d_H = from_nested_to_3d_numpy(X_train_H)
X_test_3d_H = from_nested_to_3d_numpy(X_test_H)

print(X_train_3d_H.shape)
print(X_test_3d_H.shape)

(500, 3, 152)
(500, 3, 152)


In [30]:
X_train_H = transform_data(X_train_3d_H, 10, [3], 1)
X_test_H = transform_data(X_test_3d_H, 10, [3], 1)

filter of size  3 * 122 was created

filter of size  3 * 122 was created



In [31]:
run_baseline_l2_logistic_regression(X_train_H, y_train_H, X_test_H, y_test_H)

train_acc: 0.072000 
train_AUC: 0.589020 
test_acc: 0.034000 
test_AUC: 0.553107 


In [32]:
C_list = [1.0, 0.1, 0.01, 0.001]
kernel_list = ['linear', 'poly', 'rbf', 'sigmoid']
d_list = [2, 3]
df_list = ['ovr', 'ovo']

In [33]:
write_SVM_CV_output('handwriting_svm_cv_results.txt', C_list, kernel_list, d_list, df_list, X_train_H, y_train_H, X_test_H, y_test_H)

In [34]:
find_best_SVM_models(C_list, kernel_list, d_list, df_list, X_train_H, y_train_H, X_test_H, y_test_H)

([0.01, 'poly', 3, 'ovo', 0.24, 0.082],
 [1.0, 'poly', 2, 'ovr', 0.7412776724218353, 0.6056194482463202],
 [1.0, 'poly', 3, 'ovo', 0.30548493431510154, 0.07014694657357551])